In [1]:
import os
import time

In [2]:
os.environ['KERAS_BAXKEND']='tensorflow'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import keras
#import keras_nlp
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


In [4]:
#keras.mixed_precision.set_global_policy("mixed_float16")
print(tf.__version__)
#tf.compat.v1.disable_eager_execution()

2.16.2


In [5]:
root = '/Users/pals/MICS/MICS_207/Project/datasets'
wikitext2_root = root + '/kaggle-wikitext/wikitext-2/'
train_file = wikitext2_root + 'wiki.train.tokens'
test_file  = wikitext2_root + 'wiki.test.tokens'
valid_file = wikitext2_root + 'wiki.valid.tokens'
unittest_file = wikitext2_root + 'unittest.tokens'

In [6]:
BATCH_SIZE = 8
NUM_EPOCHS = 1
SEQUENCE_LENGTH = 512
SHUFFLE_SIZE = 128
#BLOCK_SIZE = 128

In [7]:
# Encode input text using the tokenizer provided
def encode(text, tokenizer, max_length=SEQUENCE_LENGTH):
    tokens = tokenizer.encode(text, padding='max_length', truncation=True, max_length=max_length)
    input_ids = tf.constant(tokens, dtype=tf.int32)
    attention_mask = tf.constant([1 if token != tokenizer.pad_token_id else 0 for token in tokens], dtype=tf.int32)
    return input_ids, attention_mask

In [8]:
def load_dataset(file_path, tokenizer, max_length=SEQUENCE_LENGTH):
    lines = []
    with open(file_path, 'r', encoding='utf-8') as fd:
        for line in fd:
            sline = line.strip()
            if len(sline) > 0:
                lines.append(sline)
    input_ids, attention_masks = zip(*[encode(line, tokenizer, max_length=max_length) for line in lines])
    input_ids = tf.stack(input_ids)
    attention_masks = tf.stack(attention_masks)
    return tf.data.Dataset.from_tensor_slices((input_ids, attention_masks))

In [9]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
#gpt2_tokenizer.padding_side = 'left'


In [10]:
train_dataset = load_dataset(train_file, gpt2_tokenizer).shuffle(buffer_size=SHUFFLE_SIZE).batch(BATCH_SIZE)
test_dataset  = load_dataset(test_file, gpt2_tokenizer).batch(BATCH_SIZE)
valid_dataset = load_dataset(valid_file, gpt2_tokenizer).batch(BATCH_SIZE)
unittest_dataset = load_dataset(unittest_file, gpt2_tokenizer).batch(BATCH_SIZE)

In [11]:
for ele in unittest_dataset:
    print(ele)

(<tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[15496,  2159,    13, ..., 50256, 50256, 50256],
       [49488,   318,  2901, ..., 50256, 50256, 50256],
       [34784,  1365,  1110, ..., 50256, 50256, 50256]], dtype=int32)>, <tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>)


2024-07-04 09:23:14.018793: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
print(unittest_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 512), dtype=tf.int32, name=None), TensorSpec(shape=(None, 512), dtype=tf.int32, name=None))>


In [13]:
model_name = 'gpt2'
#model_name = "./gpt2_saved_v1"
gpt2_lm = TFGPT2LMHeadModel.from_pretrained(model_name)
gpt2_lm.resize_token_embeddings(len(gpt2_tokenizer)) 

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [14]:
gpt2_lm.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  124439808 
 er)                                                             
                                                                 
Total params: 124439808 (474.70 MB)
Trainable params: 124439808 (474.70 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
'''
optimizer = DPKerasAdamOptimizer(
    l2_norm_clip=1.0,
    noise_multiplier=1.1,
    num_microbatches=8,
    learning_rate=1e-4
)
'''
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4)
'''
optimizer + tf.keras.optimizers.SGD(learning_rate = 0.01,
                                momentum=0.0, 
                                nesterov=False, 
                                name='SGD')
'''                                
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [16]:
gpt2_lm.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [17]:
def generate(input_text, max_length=256):
    input_ids = gpt2_tokenizer.encode(input_text, return_tensors='tf')
    attention_mask = tf.ones_like(input_ids)
    outputs   = gpt2_lm.generate(input_ids, max_length = max_length, attention_mask=attention_mask, 
                                 num_return_sequences=5,
                                 no_repeat_ngram_size=2,  # Avoid repeating n-grams of size 2
                                 top_k=25,  # Limits the sampling pool to top_k tokens
                                 do_sample=True,
                                 temperature = 10.0,
                                 #top_p=0.95, 
                                 pad_token_id=gpt2_tokenizer.eos_token_id)
    gen_text  = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return gen_text

In [18]:
print(generate('I went on a trip to see Tajmahal in Agra. My trip was'))

I went on a trip to see Tajmahal in Agra. My trip was quite different but the journey made me remember one aspect when they visited here in Mumbai – I couldn'sn´l meet Taj Mahahm – which I have forgotten the very thing I went so many weeks. So we went from Saraj to Nagari from then onward to go to Jain as Taj has no such highways from that era". (Chermish) He is trying a much smaller pilgrimage this past summer than the first half-year. However one person was not surprised about being told he does now live there too. The last Taj is now going out into Bijaji to do his last solo performance from this stage on May 15 here and at least his audience may enjoy watching some other artists get to experience something special through their instruments and melodies (as there does exist for us a band named Srinivas) as a last few guests come through our back hall during his next concert in May 2014 as we had not come close after just this time for several minutes when Taj mutes it out with two o

In [19]:
st = time.time()
history = gpt2_lm.fit(
    train_dataset,
    epochs=NUM_EPOCHS,
    validation_data=valid_dataset
)
en = time.time()

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2971/2971 [==============================] - 22739s 8s/step - loss: 0.0072 - accuracy: 0.9991 - val_loss: 4.4741e-07 - val_accuracy: 1.0000


In [ ]:
loss, accuracy = gpt2_lm.evaluate(test_dataset)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

In [20]:
print(f'Training time {(en-st)/3600} hours')

Training time 6.316470919516352 hours


In [21]:
print(generate('I went on a trip to see Tajmahal in Agra. My trip was'))

I went on a trip to see Tajmahal in Agra. My trip was" "H",A".!""B(s"M"-D""I."I,"L"!"I wasAA["[""!"","S",""I"..."""!L:"".I"!AH.""s"[""".",S#1"H"'DHe"-""!!"L(I>""A"P"?D..."""..."(",I")He"He"...The","[!""(S(!"!"A,""-)"-"?""".In("...D(...">"#H)"?","I()""!He"…."The"!":","-!"
IThe"(AThe"...""- "!""...!!"He".".""."D,""(?"L-A",P"[."H".K"...""!The"WK"[WL." "",",M"";!""[H,"F",L,"D,"YK:"II)"";@:""(W"("#"-"(L!"."". ""-P"-I.""" ( "".-"(",""@"-:"","(."L"("[i"[


In [ ]:
gpt2_lm.save_pretrained("./gpt2_saved_v1")